In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split

with open('winequality-white.csv', 'r') as f:
  temp = np.genfromtxt(f,delimiter=',', skip_header=1)
  X = temp[:,:-1]
  y = temp[:,-1]

/Users/admin/GitHub/nn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
from sklearn.preprocessing import OneHotEncoder

y = y.reshape((-1,1))
print(y.shape)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y)
#print(enc.transform(y).toarray())
y_enc = enc.transform(y).toarray()
print(y_enc.shape)

(4898, 1)
(4898, 7)
